In [ ]:
import JupyterNotebooksLib as slicernb

In [ ]:
import slicer

# slicer.util.pip_install('xarray')
# slicer.util.pip_install('dask')
# slicer.util.pip_install('dask-image')
# slicer.util.pip_install('fastscript')
# slicer.util.pip_install('tqdm')
# slicer.util.pip_install('Pillow')
# slicer.util.pip_install('moviepy')
# slicer.util.pip_install('zarr')
# slicer.util.pip_install('netCDF4')

In [ ]:
import os
from pathlib import Path

import ctk

try:
    import matplotlib
    import matplotlib_scalebar
except ModuleNotFoundError:
    pip_install("matplotlib")
    pip_install("matplotlib-scalebar")
    import matplotlib

matplotlib.use("Agg")
import matplotlib.pyplot as plt

import microtom

# work_dir é a pasta examples do repositório
work_dir = Path(os.path.abspath("")) / "data"
print(work_dir)

# Porosimetria
O arquivo porosimetry.py contém funções uteis para determinar a distribuição de tamanho de poros e gargantas.

In [ ]:
from microtom import psd, micp, hpsd, psd_red_image, connected_image, micp_red_image

A função `psd` determina a distribuição de tamanho de poros de uma imagem. Esta função se define como a fração dos poros segmentados de uma imagem que está incluída em uma esfera de determinado diâmetro.

In [ ]:
ds = microtom.read_raw_file(work_dir + "/SINTETICA_III_LIMPA_B1_BIN_0100_0100_0100_04000nm.raw");

In [ ]:
red_saturation_psd, list_radius_psd, psd_image_data = psd(ds["bin"].data, full_image=True)

A opção full_image é opcional e igual a `False`. Se colocada igual a true, gera uma imagem da distribuição de tamanho de poros da amostra, conforme mostrado na figura abaixo.

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(20, 20))
nslice = 90
ax[0].imshow(ds["bin"].data[nslice, :, :])
ax[1].imshow(psd_image_data[nslice, :, :])
fig.tight_layout()
slicernb.MatplotlibDisplay(plt)

Muitas vezes é mais comum fazer o gráfico da saturação em relação a 1/R, como segue:

In [ ]:
blue_saturation_psd = 1 - red_saturation_psd  # Esse é o fluido que ficaria nos menores poros
plt.scatter(1 / list_radius_psd, blue_saturation_psd)
slicernb.MatplotlibDisplay(plt)

Para investigar melhor o que está acontecendo em cada ponto desta curva, pode-se utilizar a função `psd_red_image`, que retorna a imagem das regiões em que uma esfera de um determinado diâmetro conseguiu acessar o meio poroso. Exemplo: 

In [ ]:
red_image_psd = psd_red_image(ds["bin"].data, 10.0)

In [ ]:
fig, ax = plt.subplots(ncols=3, nrows=1, figsize=(20, 20))
nslice = 90
ax[0].imshow(ds["bin"].data[nslice, :, :])
ax[1].imshow(red_image_psd[nslice, :, :])
ax[2].imshow(ds["bin"].data[nslice, :, :] - red_image_psd[nslice, :, :])
slicernb.MatplotlibDisplay(plt)

In [ ]:
red_saturation_micp, list_radius_micp, micp_image_data = micp(ds["bin"].data, full_image=True)

A opção full_image é opcional e igual a `False`. Se colocada igual a true, gera uma imagem da distribuição de tamanho de volumes conectados por diâmetros de determinado tamanho da amostra, conforme mostrado na figura abaixo.

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(20, 20))
nslice = 90
ax[0].imshow(ds["bin"].data[nslice, :, :])
ax[1].imshow(micp_image_data[nslice, :, :])
slicernb.MatplotlibDisplay(plt)

Muitas vezes é mais comum fazer o gráfico da saturação em relação a 1/R, como segue:

In [ ]:
blue_saturation_micp = 1 - red_saturation_micp  # Esse é o fluido que ficaria nos menores poros
plt.scatter(1 / list_radius_micp, blue_saturation_micp)
slicernb.MatplotlibDisplay(plt)

In [ ]:
plt.scatter(1 / list_radius_micp, blue_saturation_micp)
plt.scatter(1 / list_radius_psd, blue_saturation_psd)
slicernb.MatplotlibDisplay(plt)

Existe ainda um terceiro tipo de distribuição de tamanho de poros, chamada de hierárquica, por não permitir que as esferas se sobreponham no espaço poroso.

In [ ]:
red_saturation_hpsd, list_radius_hpsd, hpsd_image_data = hpsd(ds["bin"].data, full_image=True)

In [ ]:
nslice = 90
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(20, 20))
ax[0].imshow(ds["bin"].data[nslice, :, :])
ax[1].imshow(hpsd_image_data[nslice, :, :])
slicernb.MatplotlibDisplay(plt)

In [ ]:
hpsdVolumeNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode", "HPSD")
hpsdVolumeNode.CreateDefaultDisplayNodes()

slicer.util.updateVolumeFromArray(hpsdVolumeNode, hpsd_image_data)

In [ ]:
slicernb.AppWindow.setWindowSize(scale=1.0)
slicernb.ViewInteractiveWidget()

In [ ]:
slicernb.showVolumeRendering(volume)